### Open the files
Select the 0th index counts file to be read in

In [45]:
from t3r_read import t3r
from tdms_read import tdms
import os;
from tkinter import filedialog
from zoom import zoom_factory;
import numpy as np;
from matplotlib.widgets import Slider,SpanSelector, Button
import matplotlib.pyplot as plt


%load_ext autoreload
%autoreload 2

file_path = filedialog.askopenfilename();
filename = os.path.abspath(file_path);

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
from plotter import Plotter
from dataclasses import dataclass

from utils import index_suffix, make_extension, AbstractFile


use_inputsgn_files = True
file_interval = (0,4)  # (n,m) open file with indices starting at n up to m-1
seconds_in_file = 10  # use this value if it is desired to override standard calculation of file length

abstractfile = AbstractFile(filename, index_suffix, 4)
# abstractfile = AbstractFile(filename, make_extension, '40')
    
plotter = Plotter(abstractfile, file_interval=file_interval, bin=10e-3)
seconds_in_file = plotter.seconds_in_file

In [48]:
# qt backend (opens separate window). Small bugs 
# import IPython

# shell = IPython.get_ipython()
# shell.enable_matplotlib(gui='qt')

# IPython widget backend
%matplotlib widget

plotter.set_file_interval(0, 4

plt.close('all')
plotter.init_figure()
plotter.plot_counts()
plotter.plot_input(10e-3, 1.0/8, yoffset=-8, log_offset=0.32)
plt.show()

<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>


In [37]:
# Calculate and plot the Fourier Spectrum
plotter.fft(1e-4) 

In [ ]:
# Disable the input and log graphs
plotter.set_display_input(True)

In [ ]:
# Hide the counts plot
plotter.main_ax.set_visible(False)

### Find events and plot them

As of now the events are detected by the DIO value going from True to False

The timetraces near each event are loaded in, plotted and the graph is then saved as .png in a folder in the TDMS file's directory

In [49]:
from utils import make_extension, DIO_event, read_interval_from_multiple_files, AbstractCountsFile, COUNTS
        
       
event_bins = 10e-3    
    
# loop over events to find thresholding events
dio_events = []
for i in plotter.log_entries:
    if i.previous_event and i.previous_event._dio:
        if not i._dio:
            t = i.t_start 
            file_index = int(t // seconds_in_file)
            relative_to_file = t - file_index*seconds_in_file
            dio_events.append(DIO_event(file_index=file_index, 
                                        seconds_in_file=relative_to_file, total_seconds = t))


print('In this experiment there were %i observed encounters' % len(dio_events))

# draw counts near DIO events

left_margin = 6
right_margin = 4

event_fig = plt.figure()
for event in dio_events:
    event_data_x, event_data_y = read_interval_from_multiple_files(AbstractCountsFile(abstractfile), seconds_in_file,
                                                                  event.total_seconds-left_margin, event.total_seconds+right_margin,
                                                                  10e-3)
    
    save_dir = abstractfile.dir + ('events_%s%s\\' % (abstractfile.basename, COUNTS))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    event_fig.clf()
    plt.plot(event_data_x + int((event.total_seconds-left_margin)//seconds_in_file)*seconds_in_file, event_data_y / event_bins)
    plt.scatter(event.total_seconds, 0, color='r')
    plt.savefig(save_dir + ('%i' %event.total_seconds) + '.png')
    
plt.close(event_fig)
    

In this experiment there were 16 observed encounters
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>


Traceback (most recent call last):
  File "C:\Users\lion\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 537, in _hover_handler
    self._on_select_event(event)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 579, in _on_select_event
    self.add_selection(pi)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 384, in add_selection
    cb(sel)
  File "D:\Scripts\t3r-data-reader\plotter.py", line 121, in on_add
AttributeError: 'numpy.intc' object has no attribute 'int'


<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>


Traceback (most recent call last):
  File "C:\Users\lion\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 537, in _hover_handler
    self._on_select_event(event)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 579, in _on_select_event
    self.add_selection(pi)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 384, in add_selection
    cb(sel)
  File "D:\Scripts\t3r-data-reader\plotter.py", line 121, in on_add
AttributeError: 'numpy.intc' object has no attribute 'int'


<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>


...

<class 'mplcursors._pick_info.Selection'>


Traceback (most recent call last):
  File "C:\Users\lion\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 537, in _hover_handler
    self._on_select_event(event)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 579, in _on_select_event
    self.add_selection(pi)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 384, in add_selection
    cb(sel)
  File "D:\Scripts\t3r-data-reader\plotter.py", line 121, in on_add
AttributeError: 'numpy.intc' object has no attribute 'int'
Traceback (most recent call last):
  File "C:\Users\lion\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 537, in _hover_handler
    self._on_select_event(event)
  File "C:\Users\

<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>


Traceback (most recent call last):
  File "C:\Users\lion\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 537, in _hover_handler
    self._on_select_event(event)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 579, in _on_select_event
    self.add_selection(pi)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 384, in add_selection
    cb(sel)
  File "D:\Scripts\t3r-data-reader\plotter.py", line 121, in on_add
AttributeError: 'numpy.intc' object has no attribute 'int'


<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>


In [51]:
from utils import read_interval_from_multiple_files


index = 10 # open event with this index (starting at 0)
lm = 6 # left margin
rm = 4 # right margin
fft_bin = 1e-3
binsize = 10e-3

event = dio_events[index]

fig, axes = plt.subplots(3, figsize=(8,8))

def onselect(xmin, xmax):
    axes[1].clear()
    axes[2].clear()
    zoomx, zoomy = read_interval_from_multiple_files(AbstractCountsFile(abstractfile), seconds_in_file,
                                                                  xmin, xmax, binsize)
    
    axes[1].plot(zoomx, zoomy / binsize)

    t, s = read_interval_from_multiple_files(AbstractCountsFile(abstractfile), seconds_in_file,
                                                                  xmin, xmax, fft_bin)
    
    fft_y = np.abs(np.fft.fft(s))
    fft_x = np.arange(0, np.shape(fft_y)[0], 1) / fft_bin / np.shape(s)[0];
    
    axes[2].loglog(fft_x, fft_y)

datax, datay = read_interval_from_multiple_files(AbstractCountsFile(abstractfile), seconds_in_file,
                                                                    event.total_seconds-lm, event.total_seconds+rm,
                                                                    binsize)

axes[0].plot(datax + int((event.total_seconds-lm)//seconds_in_file)*seconds_in_file, datay / binsize)
event_span = SpanSelector(axes[0], onselect, 'horizontal', useblit=True,
                                rectprops=dict(alpha=0.7, facecolor='orange'), span_stays=True, minspan=0.05)


Traceback (most recent call last):
  File "C:\Users\lion\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 537, in _hover_handler
    self._on_select_event(event)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 579, in _on_select_event
    self.add_selection(pi)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 384, in add_selection
    cb(sel)
  File "D:\Scripts\t3r-data-reader\plotter.py", line 121, in on_add
AttributeError: 'numpy.intc' object has no attribute 'int'


<class 'mplcursors._pick_info.Selection'>


Traceback (most recent call last):
  File "C:\Users\lion\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 537, in _hover_handler
    self._on_select_event(event)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 579, in _on_select_event
    self.add_selection(pi)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 384, in add_selection
    cb(sel)
  File "D:\Scripts\t3r-data-reader\plotter.py", line 121, in on_add
AttributeError: 'numpy.intc' object has no attribute 'int'


<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Sele

Traceback (most recent call last):
  File "C:\Users\lion\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 537, in _hover_handler
    self._on_select_event(event)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 579, in _on_select_event
    self.add_selection(pi)
  File "C:\Users\lion\anaconda3\lib\site-packages\mplcursors\_mplcursors.py", line 384, in add_selection
    cb(sel)
  File "D:\Scripts\t3r-data-reader\plotter.py", line 121, in on_add
AttributeError: 'numpy.intc' object has no attribute 'int'


<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Selection'>
<class 'mplcursors._pick_info.Sele